In [2]:
import torchcraft
from torchcraft import replayer
import os
import pickle
import numpy as np
import data_utils
from generate_role_datasets import unit_to_dict, game_over_time, post_process, hmm_data


In [3]:
replays_master = open('good_files.txt', 'r').readlines()
for i in range(len(replays_master)):
    replays_master[i] = replays_master[i].split('\n')[0]

In [4]:
num_games_to_parse = 1
step_size = 3
window_size = 20
# I only want information on these units:
terran_valid_types = [0, 1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 30, 32, 34, 58]
zerg_valid_types = [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 62, 103]
protoss_valid_types = [60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 83, 84, 85]
zerg_babies = [35, 36, 59, 97]
valid_units = terran_valid_types  # + zerg_valid_types + protoss_valid_types


In [ ]:
# Cell for data generation across many games
all_games = []
for replay_path in replays_master:
    if len(all_games) >= num_games_to_parse:
        break
    game_info = game_over_time(replay_path, valid_types=valid_units, playerid=2, step_size=step_size)
    if game_info != -1:
        all_games.append(game_info)
        hmm_samples = hmm_data(game_info, n_timesteps=window_size)
        print("Length of game: %s" % len(game_info))
        print("Number of HMM Samples: %s" % len(hmm_samples))
        X = []
        for i in range(0, len(hmm_samples), 20):
            X.append(hmm_samples[i])


In [ ]:
# Cell for data generation across 1 game (debugging)
replay_path = '/media/asilva/HD_home/StarData/dumped_replays/0/bwrep_poa7y.tcr'
game_info = game_over_time(replay_path, valid_types=valid_units, playerid=0, step_size=3)
hmm_samples = hmm_data(game_info, n_timesteps=window_size)
print("Length of game: %s" % len(game_info))
print("Number of HMM Samples: %s" % len(hmm_samples))
X = hmm_samples

In [ ]:
# print len(processed)
print len(game_info)
print len(X)
print len(X[0])
print X[-100]